In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from collections import Counter
from torch.optim import Adam
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os

## Dataset

In [2]:
human_faces_dir = '/kaggle/input/human-faces/Humans'
flowers_dir = '/kaggle/input/flowers-dataset/test'

human_faces_files = len(os.listdir(human_faces_dir))
flowers_files = len(os.listdir(flowers_dir))

print(f"Number of files in human_faces_dir: {human_faces_files}")
print(f"Number of files in flowers_dir: {flowers_files}")

Number of files in human_faces_dir: 7219
Number of files in flowers_dir: 924


In [3]:
my_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

class CustomDataset(Dataset):
    def __init__(self, human_faces_dir, flowers_dir, transform=None):
        self.human_faces_dir = human_faces_dir
        self.flowers_dir = flowers_dir
        
        human_faces_images = [os.path.join(human_faces_dir, fname) for fname in os.listdir(human_faces_dir)]
        self.human_faces_images = random.sample(human_faces_images, 300)
        flowers_images = [os.path.join(flowers_dir, fname) for fname in os.listdir(flowers_dir)]
        self.flowers_images = random.sample(flowers_images, 200)
        
        self.all_images = self.human_faces_images + self.flowers_images
        self.labels = [1] * len(self.human_faces_images) + [0] * len(self.flowers_images)
        self.transform = transform

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_path = self.all_images[idx]
        image = Image.open(img_path).convert("RGB")
        
        label = self.labels[idx]
        
        if self.transform: image = self.transform(image)
        return image, label

dataset = CustomDataset(human_faces_dir, flowers_dir, transform=my_transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
for images, labels in dataloader:
    print(images.shape)
    print(labels)
    break

torch.Size([128, 3, 128, 128])
tensor([1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 0])


In [5]:
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = test_dir
        test_images = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(('.jpg', '.png', '.jpeg'))]
        test_images = random.sample(test_images, 128)
        self.all_images = test_images
        self.transform = transform

    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        img_path = self.all_images[idx]
        image = Image.open(img_path).convert("RGB")
        
        if self.transform: image = self.transform(image)
        return image

test_dir = "/kaggle/input/face-dataset/human-swap/"
test_dataset = TestDataset(test_dir, transform=my_transform)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [6]:
for images in test_dataloader:
    print(images.shape)
    break

torch.Size([128, 3, 128, 128])


## CNN

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 256, kernel_size=9, stride=1, padding=4)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=9, stride=2, padding=4)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=9, stride=2, padding=4)
        self.conv4 = nn.Conv2d(64, 32, kernel_size=9, stride=2, padding=4)
        self.fc_input_size = self._get_fc_input_size()
        self.fc = nn.Linear(self.fc_input_size, 2)
        self.softmax = nn.Softmax(dim=1)

    def _get_fc_input_size(self):
        dummy_input = torch.zeros(1, 3, 128, 128)
        x = self.conv1(dummy_input)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv2(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        return x.size(1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv2(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv3(x))
        x = F.relu(F.max_pool2d(x, 2))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.softmax(x)
        return x

    def loss(self, output, target):
        return F.cross_entropy(output, target)

In [8]:
cnn = CNN().to(device)
optimizer = Adam(cnn.parameters())

n_epochs = 20
for epoch in range(n_epochs):
    cnn.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    for batch_id, (data, target) in enumerate(tqdm(dataloader)):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = cnn(data)
        loss = cnn.loss(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, preds = torch.max(output, 1)
        correct_train += preds.eq(target).sum().item()
        total_train += target.size(0)
    
    train_accuracy = correct_train / total_train
    avg_train_loss = train_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_train_loss:.4f} - Accuracy: {train_accuracy:.4f}")

100%|██████████| 4/4 [00:20<00:00,  5.11s/it]


Epoch 1/20 - Loss: 0.6972 - Accuracy: 0.5200


100%|██████████| 4/4 [00:13<00:00,  3.47s/it]


Epoch 2/20 - Loss: 0.6059 - Accuracy: 0.6980


100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


...

Epoch 18/20 - Loss: 0.3554 - Accuracy: 0.9580


100%|██████████| 4/4 [00:13<00:00,  3.40s/it]


Epoch 19/20 - Loss: 0.3643 - Accuracy: 0.9440


100%|██████████| 4/4 [00:13<00:00,  3.35s/it]

Epoch 20/20 - Loss: 0.3496 - Accuracy: 0.9620


In [9]:
cnn.eval()
correct_test = 0
total_test = 0

for data in test_dataloader:
    data = data.to(device)
    output = cnn(data)
    target = torch.zeros(output.size(0)).float()
    target = target.to(device)

    _, preds = torch.max(output, 1)
    correct_test += preds.eq(target).sum().item()
    total_test += target.size(0)

test_accuracy = correct_test / total_test
print(f"Accuracy: {test_accuracy:.4f}")

Accuracy: 0.0078
